In [2]:
import pandas as pd


books = pd.read_csv('books_with_categories.csv')

In [4]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device="cuda")
classifier("I love this!")


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

C:\Users\vamsi7\anaconda3\envs\pyTchGPU\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vamsi7\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

C:\Users\vamsi7\anaconda3\envs\pyTchGPU\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\vamsi7\anaconda3\envs\pyTchGPU\lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528676815330982},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419781267642975},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385195800103247}]]

In [6]:
#sentence wise sentiment analysis
prediction = classifier(books["description"][0].split("."))
prediction

[[{'label': 'surprise', 'score': 0.7296026945114136},
  {'label': 'neutral', 'score': 0.14038574695587158},
  {'label': 'fear', 'score': 0.06816213577985764},
  {'label': 'joy', 'score': 0.04794243350625038},
  {'label': 'anger', 'score': 0.009156356565654278},
  {'label': 'disgust', 'score': 0.002628471702337265},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937047362327576},
  {'label': 'disgust', 'score': 0.2735918462276459},
  {'label': 'joy', 'score': 0.109082892537117},
  {'label': 'sadness', 'score': 0.09362728148698807},
  {'label': 'anger', 'score': 0.04047832638025284},
  {'label': 'surprise', 'score': 0.026970166712999344},
  {'label': 'fear', 'score': 0.0068790502846241}],
 [{'label': 'neutral', 'score': 0.6462167501449585},
  {'label': 'sadness', 'score': 0.24273262917995453},
  {'label': 'disgust', 'score': 0.04342260956764221},
  {'label': 'surprise', 'score': 0.028300534933805466},
  {'label': 'joy', 'score': 0.014211485162

In [9]:
sorted(prediction[0],key= lambda x:x['label'])

[{'label': 'anger', 'score': 0.009156356565654278},
 {'label': 'disgust', 'score': 0.002628471702337265},
 {'label': 'fear', 'score': 0.06816213577985764},
 {'label': 'joy', 'score': 0.04794243350625038},
 {'label': 'neutral', 'score': 0.14038574695587158},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296026945114136}]

In [15]:
import numpy as np
from tqdm import tqdm

In [12]:
emotion_labels = ["anger","disqust","fear","joy","sadness","surprise","neutral"]
isbn = []

emotion_scores = {label:[] for label in emotion_labels}

def calculate_max_emotion_score(predictions):
    per_emotion_scores = {label:[] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction,key=lambda x: x['label'])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_prediction[index]['score'])
    return {label:np.max(scores) for label,scores in per_emotion_scores.items()}

In [13]:
for i in range(10):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
emotion_scores

{'anger': [0.0641336441040039,
  0.612619161605835,
  0.0641336441040039,
  0.35148346424102783,
  0.08141231536865234,
  0.23222459852695465,
  0.5381839275360107,
  0.0641336441040039,
  0.3006700575351715,
  0.0641336441040039],
 'disqust': [0.2735918462276459,
  0.34828469157218933,
  0.10400668531656265,
  0.150722473859787,
  0.18449503183364868,
  0.7271751165390015,
  0.15585486590862274,
  0.10400668531656265,
  0.2794809639453888,
  0.17792744934558868],
 'fear': [0.9281679391860962,
  0.9425276517868042,
  0.9723208546638489,
  0.3607070744037628,
  0.09504338353872299,
  0.05136282369494438,
  0.7474286556243896,
  0.4044956862926483,
  0.9155240654945374,
  0.05136282369494438],
 'joy': [0.9327971935272217,
  0.704420804977417,
  0.7672367691993713,
  0.2518812119960785,
  0.040564414113759995,
  0.04337586089968681,
  0.8725652098655701,
  0.040564414113759995,
  0.040564414113759995,
  0.040564414113759995],
 'sadness': [0.6462167501449585,
  0.8879395127296448,
  0.5494

In [16]:
emotion_labels = ["anger","disqust","fear","joy","sadness","surprise","neutral"]
isbn = []

emotion_scores = {label:[] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:43<00:00, 31.82it/s]


In [19]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df['isbn13'] = isbn

emotion_df

,anger,disqust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646217,0.967157,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765,9788185300535


In [20]:
books = pd.merge(books,emotion_df,on='isbn13')

books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disqust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932797,0.646217,0.967157,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765


In [21]:
books.to_csv('books_with_emotions.csv',index=False)